#### Train DT network without k-fold cross validation

In [ ]:
import sys
sys.path.append("..")

from components.fcn import FCN
from components.datasets import ConesDataset

dataset_name = "dataset_no_cross_offline"

train_dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/train", train=True, convert_to_dt=True, pre_generated=True, augment_rotate=True)
train_dataloader = train_dataset.get_dataloader(shuffle=True,batch_size=32)

validation_dataset = ConesDataset(f"F:/Data/ConeDatasets/{dataset_name}/validation", train=False, convert_to_dt=True, pre_generated=True)
validation_dataloader = validation_dataset.get_dataloader(shuffle=False, batch_size=32)

fcn = FCN(train_dataloader, validation_dataloader=validation_dataloader, lr=3e-4, depth=3, input_channels=1, 
    initial_feature_maps=32, blocks_per_resolution_layer=2, use_MSE_loss=True, masked_MSE_loss=True)

fcn.train_network(epochs=100)

### Train DT network with k-fold cross validation (used for default dataset)

In [ ]:
import sys
import os
sys.path.append("..")

from components.fcn import FCN
from components.datasets import ConesDataset

# Config
# number of folds
k = 8
# number of epochs per fold
epochs_per_fold = 300
# dataset name
dataset_name = "20230301_cross_eroded_equalized_64_cropped"

# Prepare data for training
path_to_training_dataset = f"F:/Data/ConeDatasets/{dataset_name}/train"
dataset = ConesDataset(path_to_training_dataset, train=True, 
    convert_to_dt=True, #False,
    convert_to_gaussians=False, #True,
    pre_generated=True, 
    augment_rotate=True, 
    augment_blur=False, #True
    augment_elastic=False,
    adaptive_dilation=False,
    dilation = 1) 

# Create folds and save to disk (ONLY USE ONCE)
path_k_folds = "F:/Repositories/cone-detection-master-thesis/folds"
if not os.path.exists(path_k_folds):
    dataset.get_k_fold_splits(k, path_k_folds)

# # Train with a SINGLE fold
# fold_id = 0
# fold_path = f"F:/Repositories/cone-detection-master-thesis/folds/fold_{fold_id}.npz"
# train_dataloader, validation_dataloader = dataset.get_single_dataloader_cross_validation(fold_path, batch_size=32)
# FCN.train_network_single_cross_validation(fold_id, train_dataloader, validation_dataloader, max_epochs=epochs_per_fold, early_stopping=True)

# Train WITH all k folds
dataloaders = dataset.get_dataloaders_k_fold_cross_validation(k, batch_size=32)
FCN.train_networks_k_fold_cross_validation(dataloaders, max_epochs=epochs_per_fold, lr=3e-4, early_stopping=False,
                                           use_lr_scheduler=True)